In [116]:
import pandas as pd



In [117]:
open_line_list = pd.read_csv('COVID19_open_line_list.csv')

#Rimuovo le colonne in più vuote, caricate da pandas
open_line_list = open_line_list.iloc[:,0:33]

#Recupero solamente le colonne di interesse
open_line_list = open_line_list[['age','sex','city','province','country','wuhan(0)_not_wuhan(1)','date_confirmation','symptoms','lives_in_Wuhan','travel_history_location','chronic_disease_binary','chronic_disease','outcome','date_death_or_discharge']]

#La colonna Chronic_disease_binary viene popolata con 0 dove i valori sono na, in quanto corrispondono ad una mancata presenza di malattie croniche
open_line_list['chronic_disease_binary']=open_line_list['chronic_disease_binary'].fillna(0)

#Creo una colonna binaria per individuare rapidamente se una persona è stata a Wuhan o no, in base ai dati di travel history location
open_line_list['travel_in_Wuhan']=open_line_list['travel_history_location'].str.contains("Wuhan",na=False, regex=True)

#Cambio la logica di wuhan(0)_not_wuhan(1), mettendo a 1 se è stato a wuhan, 0 altrimenti. Cambio il nome della colonna in "inWuhan"
open_line_list['wuhan(0)_not_wuhan(1)']=1-open_line_list['wuhan(0)_not_wuhan(1)']
open_line_list.rename(columns={"wuhan(0)_not_wuhan(1)":"inWuhan"},inplace=True)
open_line_list['inWuhan']=open_line_list['inWuhan'].fillna(0)

#Popolo i NaN con "no" nella colonna "lives_in_wuhan" e creo rispettiva colonna binaria
open_line_list['lives_in_Wuhan']=open_line_list['lives_in_Wuhan'].fillna('no')
open_line_list['lives_in_Wuhan_binary']=open_line_list['lives_in_Wuhan']!='no'

#Elimino le righe in cui "country" è nullo
indexOLL = open_line_list[open_line_list['country'].isnull()].index
open_line_list.drop(indexOLL,inplace=True)

#Elimino le colonne che non sono più utili, in quanto o troppo poco presenti o sostituite da colonne binarie
open_line_list.drop(columns=["lives_in_Wuhan","travel_history_location","chronic_disease","outcome","date_death_or_discharge"],inplace=True)

#Rendo vuoti i campi "age" e "date_confirmation" nei casi in cui sono presenti dei range e non valori precisi
indexesOLL = open_line_list[open_line_list['age'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexesOLL.tolist(),'age'] = ''
indexesOLL = open_line_list[open_line_list['date_confirmation'].str.contains("-",na=True, regex=None)==True].index
open_line_list.loc[indexesOLL.tolist(),'date_confirmation'] = ''

open_line_list.to_csv('COVID19_open_line_list_clean.csv',index=False)